In [1]:
import torch
import pandas as pd

In [2]:
df_tasks = pd.read_csv('../data/final_data_enhanced.csv')
df_translators = pd.read_pickle('../data/final_translators_enhanced.pkl')

/tmp/ipykernel_17810/1844511054.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  df_tasks = pd.read_csv('../data/final_data_enhanced.csv')


In [3]:
df_tasks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554029 entries, 0 to 554028
Data columns (total 29 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   Unnamed: 0                   554029 non-null  int64  
 1   PROJECT_ID                   554029 non-null  object 
 2   PM                           554029 non-null  object 
 3   TASK_ID                      554029 non-null  int64  
 4   START                        554029 non-null  object 
 5   END                          554029 non-null  object 
 6   TASK_TYPE                    554029 non-null  object 
 7   SOURCE_LANG                  554029 non-null  object 
 8   TARGET_LANG                  554029 non-null  object 
 9   TRANSLATOR                   554029 non-null  object 
 10  ASSIGNED                     554029 non-null  object 
 11  READY                        554029 non-null  object 
 12  WORKING                      554029 non-null  object 
 13 

In [4]:
columns_drop_tasks = [
    'PROJECT_ID',
    'START',
    'END',
    'ASSIGNED',
    'READY',
    'WORKING',
    'DELIVERED',
    'RECEIVED',
    'CLOSE',
    'COST',
    'MANUFACTURER',
    'MANUFACTURER_SECTOR',
    'MANUFACTURER_INDUSTRY_GROUP',
    'MANUFACTURER_SUBINDUSTRY',
    '_work_ready',
    '_time_taken',
    '_time_reception',
    '_time_to_close',
]
df_tasks.drop(columns=columns_drop_tasks, inplace=True)
df_tasks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554029 entries, 0 to 554028
Data columns (total 11 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             554029 non-null  int64  
 1   PM                     554029 non-null  object 
 2   TASK_ID                554029 non-null  int64  
 3   TASK_TYPE              554029 non-null  object 
 4   SOURCE_LANG            554029 non-null  object 
 5   TARGET_LANG            554029 non-null  object 
 6   TRANSLATOR             554029 non-null  object 
 7   HOURS                  554029 non-null  float64
 8   HOURLY_RATE            554029 non-null  int64  
 9   QUALITY_EVALUATION     554029 non-null  int64  
 10  MANUFACTURER_INDUSTRY  554024 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 46.5+ MB


In [5]:
df_translators.info()

<class 'pandas.core.frame.DataFrame'>
Index: 871 entries, 0 to 982
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   TRANSLATOR               871 non-null    object 
 1   SOURCE_LANG              871 non-null    object 
 2   TARGET_LANG              871 non-null    object 
 3   HOURLY_RATE              871 non-null    float64
 4   MANUFACTURER_INDUSTRY    871 non-null    object 
 5   TASK_TYPE                871 non-null    object 
 6   PM                       871 non-null    object 
 7   HOURS_mean               871 non-null    float64
 8   HOURLY_RATE_mean         871 non-null    float64
 9   QUALITY_EVALUATION_mean  871 non-null    float64
 10  SOURCE_LANG_EMBED        871 non-null    object 
 11  TARGET_LANG_EMBED        871 non-null    object 
 12  INDUSTRY_EMBED           815 non-null    object 
dtypes: float64(4), object(9)
memory usage: 95.3+ KB


In [6]:
positives = pd.merge(
    df_tasks,
    df_translators,
    on='TRANSLATOR',
    how='left',
    suffixes=('_task', '_translator'),
)

In [7]:
positives.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 554029 entries, 0 to 554028
Data columns (total 23 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   Unnamed: 0                        554029 non-null  int64  
 1   PM_task                           554029 non-null  object 
 2   TASK_ID                           554029 non-null  int64  
 3   TASK_TYPE_task                    554029 non-null  object 
 4   SOURCE_LANG_task                  554029 non-null  object 
 5   TARGET_LANG_task                  554029 non-null  object 
 6   TRANSLATOR                        554029 non-null  object 
 7   HOURS                             554029 non-null  float64
 8   HOURLY_RATE_task                  554029 non-null  int64  
 9   QUALITY_EVALUATION                554029 non-null  int64  
 10  MANUFACTURER_INDUSTRY_task        554024 non-null  object 
 11  SOURCE_LANG_translator            554029 non-null  o

Some renaming of the columns

In [8]:
positives.rename(
    columns={
        'TRANSLATOR': 'TRANSLATOR_NAME',
        'HOURS': 'FORECAST_task',
        'MANUFACTURER_INDUSTRY_task': 'INDUSTRY_task',
        'SOURCE_LANG_EMBED': 'SOURCE_LANG_EMBED_translator',
        'TARGET_LANG_EMBED': 'TARGET_LANG_EMBED_translator',
        'INDUSTRY_EMBED': 'INDUSTRY_EMBED_translator',

    },
    inplace=True,
)

### Language and Industry Embeddings

In [9]:
import sys
import os
from warnings import warn

notebooks_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(notebooks_dir, ".."))
sys.path.append(project_root)

from scripts.lang_embeddings import language_tokenizer
from scripts.lang_embeddings import EmbeddingLookup as LookupLang
from scripts.lang_embeddings import load_embedding_data as load_lang

from scripts.industry_embeddings import industry_tokenizer
from scripts.industry_embeddings import EmbeddingLookup as LookupIndustry
from scripts.industry_embeddings import load_embedding_data as load_industry

/home/AIPS01/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
data_embed_lang = load_lang("../scripts/language_embeddings.pkl")
data_embed_industry = load_industry("../scripts/final_industry_embeddings.pkl")

lang_lookup = LookupLang(data_embed_lang)
industry_lookup = LookupIndustry(data_embed_industry)

Successfully loaded embedding data from '../scripts/language_embeddings.pkl'
Successfully loaded embedding data from '../scripts/final_industry_embeddings.pkl'
EmbeddingLookup initialized with 79 languages.
  Original embedding dimension: 384
  Latent embedding dimension: 10
EmbeddingLookup initialized with 286 languages.
  Original embedding dimension: 384
  Latent embedding dimension: 10


In [11]:
positives = positives[positives["INDUSTRY_task"].notna()]

positives['INDUSTRY_task'] = positives['INDUSTRY_task'].apply(industry_tokenizer).apply(industry_lookup.get_vector)
positives['SOURCE_LANG_task'] = positives['SOURCE_LANG_task'].apply(language_tokenizer).apply(lang_lookup.get_vector)
positives['TARGET_LANG_task'] = positives['TARGET_LANG_task'].apply(language_tokenizer).apply(lang_lookup.get_vector)

In [12]:
columns_to_drop = [
    'TASK_ID',
    'QUALITY_EVALUATION',
    'SOURCE_LANG_translator',
    'TARGET_LANG_translator',
    'MANUFACTURER_INDUSTRY_translator',
]

positives.drop(columns=columns_to_drop, inplace=True)

In [13]:
positives

,Unnamed: 0,PM_task,TASK_TYPE_task,SOURCE_LANG_task,TARGET_LANG_task,TRANSLATOR_NAME,FORECAST_task,HOURLY_RATE_task,INDUSTRY_task,HOURLY_RATE_translator,TASK_TYPE_translator,PM_translator,HOURS_mean,HOURLY_RATE_mean,QUALITY_EVALUATION_mean,SOURCE_LANG_EMBED_translator,TARGET_LANG_EMBED_translator,INDUSTRY_EMBED_translator
0,0,PMT,Miscellaneous,"[0.049495008, -0.07142061, -0.005828036, -0.01...","[0.049495008, -0.07142061, -0.005828036, -0.01...",Victor,0.00,13,"[0.008479051, 0.060991026, 0.04450611, 0.02254...",18.000000,Engineering,BMT,0.262543,14.816151,6.855670,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.051018845, 0.013300572, 0.061036862, -0.02..."
1,1,PMT,Miscellaneous,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.08935569, -0.07945183, 0.026438605, 0.01634...",Severino,3.10,17,"[0.008479051, 0.060991026, 0.04450611, 0.02254...",14.500000,Translation,PMT,4.796947,17.003971,6.976467,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[0.005651269, 0.056426913, -0.016729122, -0.06..."
2,2,KMT,Engineering,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...",Jeronimo,1.50,11,"[0.02412469, -0.08445858, -0.015656583, 0.0488...",18.000000,ProofReading,KMT,0.035435,11.217391,6.391304,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[0.005651269, 0.056426913, -0.016729122, -0.06..."
3,3,KMT,Engineering,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.061353415, -0.025645662, 0.11521641, -0.058...",Estela,0.25,17,"[0.013856687, -0.047070704, -0.08179597, -0.04...",25.745098,Engineering,RMT,0.464443,16.316802,6.826923,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.051018845, 0.013300572, 0.061036862, -0.02..."
4,4,PMT,Management,"[0.049495008, -0.07142061, -0.005828036, -0.01...","[0.05821279, -0.058210686, 0.13881628, -0.0406...",Maria Alexandra,0.00,20,"[0.008479051, 0.060991026, 0.04450611, 0.02254...",19.428571,Engineering,BMT,0.524815,18.666667,7.074074,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.049495008, -0.07142061, -0.005828036, -0.01...","[-0.051018845, 0.013300572, 0.061036862, -0.02..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554024,554024,KMT,ProofReading,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.05821279, -0.058210686, 0.13881628, -0.0406...",Ascension,0.17,14,"[-0.044932816, -0.0038222447, 0.08252808, 0.01...",14.875000,ProofReading,KMT,0.954039,14.297622,6.975477,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.056813803, -0.089675784, -0.044186905, 0.0..."
554025,554025,KMT,ProofReading,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.05821279, -0.058210686, 0.13881628, -0.0406...",Ascension,0.11,14,"[-0.044932816, -0.0038222447, 0.08252808, 0.01...",14.875000,ProofReading,KMT,0.954039,14.297622,6.975477,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.056813803, -0.089675784, -0.044186905, 0.0..."
554026,554026,KMT,ProofReading,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.05821279, -0.058210686, 0.13881628, -0.0406...",Ascension,0.10,14,"[-0.044932816, -0.0038222447, 0.08252808, 0.01...",14.875000,ProofReading,KMT,0.954039,14.297622,6.975477,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.056813803, -0.089675784, -0.044186905, 0.0..."
554027,554027,KMT,ProofReading,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.05821279, -0.058210686, 0.13881628, -0.0406...",Ascension,0.10,14,"[-0.044932816, -0.0038222447, 0.08252808, 0.01...",14.875000,ProofReading,KMT,0.954039,14.297622,6.975477,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0

In [14]:
import pickle
from sklearn.preprocessing import OneHotEncoder

In [15]:
positives['PM_task'] = positives['PM_task'].astype('category')
positives['PM_translator'] = positives['PM_translator'].astype('category')
positives['TRANSLATOR_NAME'] = positives['TRANSLATOR_NAME'].astype('category')
positives['TASK_TYPE_task'] = positives['TASK_TYPE_task'].astype('category')
positives['TASK_TYPE_translator'] = positives['TASK_TYPE_translator'].astype('category')

In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder # Ensure other imports like StandardScaler are present if needed later

# Assume 'positives' is your initial DataFrame after the merge
# and before any encoding or scaling.

task_categorical = [
    'PM_task',
    'TASK_TYPE_task',
]

translator_categorical = [
    'PM_translator',
    'TASK_TYPE_translator',
]

task_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
translator_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

task_encoder.fit(positives[task_categorical].copy())
translator_encoder.fit(positives[translator_categorical].copy())

task_encoded_array = task_encoder.transform(positives[task_categorical].copy())
translator_encoded_array = translator_encoder.transform(positives[translator_categorical].copy())

task_categorical_vector_name = 'task_categorical_vector'
positives[task_categorical_vector_name] = pd.Series(list(task_encoded_array), index=positives.index).apply(np.array)

translator_categorical_vector_name = 'translator_categorical_vector'
positives[translator_categorical_vector_name] = pd.Series(list(translator_encoded_array), index=positives.index).apply(np.array)

columns_to_drop = task_categorical + translator_categorical + ['Unnamed: 0', 'TRANSLATOR_NAME']
positives.drop(columns=columns_to_drop, inplace=True, errors='ignore')

In [17]:
positives.info()

<class 'pandas.core.frame.DataFrame'>
Index: 554024 entries, 0 to 554028
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   SOURCE_LANG_task               554012 non-null  object 
 1   TARGET_LANG_task               554019 non-null  object 
 2   FORECAST_task                  554024 non-null  float64
 3   HOURLY_RATE_task               554024 non-null  int64  
 4   INDUSTRY_task                  554024 non-null  object 
 5   HOURLY_RATE_translator         554024 non-null  float64
 6   HOURS_mean                     554024 non-null  float64
 7   HOURLY_RATE_mean               554024 non-null  float64
 8   QUALITY_EVALUATION_mean        554024 non-null  float64
 9   SOURCE_LANG_EMBED_translator   554024 non-null  object 
 10  TARGET_LANG_EMBED_translator   554024 non-null  object 
 11  INDUSTRY_EMBED_translator      543407 non-null  object 
 12  task_categorical_vector        5540

In [18]:
from sklearn.preprocessing import StandardScaler

scalar_columns = [
    'FORECAST_task',
    'HOURS_mean',
    'HOURLY_RATE_task',
    'HOURLY_RATE_translator',
    'HOURLY_RATE_mean',
    'QUALITY_EVALUATION_mean'
]

scalar = StandardScaler()

scalar_data = positives[scalar_columns].values
scalar.fit(scalar_data)
scalar_data = scalar.transform(scalar_data)

positives[scalar_columns] = scalar_data

In [19]:
positives

,SOURCE_LANG_task,TARGET_LANG_task,FORECAST_task,HOURLY_RATE_task,INDUSTRY_task,HOURLY_RATE_translator,HOURS_mean,HOURLY_RATE_mean,QUALITY_EVALUATION_mean,SOURCE_LANG_EMBED_translator,TARGET_LANG_EMBED_translator,INDUSTRY_EMBED_translator,task_categorical_vector,translator_categorical_vector
0,"[0.049495008, -0.07142061, -0.005828036, -0.01...","[0.049495008, -0.07142061, -0.005828036, -0.01...",-0.347371,-0.649924,"[0.008479051, 0.060991026, 0.04450611, 0.02254...",0.010854,-0.921160,-0.330310,-0.811840,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.051018845, 0.013300572, 0.061036862, -0.02...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.08935569, -0.07945183, 0.026438605, 0.01634...",0.112435,0.120736,"[0.008479051, 0.060991026, 0.04450611, 0.02254...",-0.975491,1.087270,0.133758,-0.342186,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[0.005651269, 0.056426913, -0.016729122, -0.06...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...",-0.124884,-1.035254,"[0.02412469, -0.08445858, -0.015656583, 0.0488...",0.010854,-1.021754,-1.093660,-2.617279,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[0.005651269, 0.056426913, -0.016729122, -0.06...","[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.061353415, -0.025645662, 0.11521641, -0.058...",-0.310290,0.120736,"[0.013856687, -0.047070704, -0.08179597, -0.04...",2.193522,-0.831732,-0.012000,-0.923608,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.051018845, 0.013300572, 0.061036862, -0.02...","[0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,"[0.049495008, -0.07142061, -0.005828036, -0.01...","[0.05821279, -0.058210686, 0.13881628, -0.0406...",-0.347371,0.698731,"[0.008479051, 0.060991026, 0.04450611, 0.02254...",0.413444,-0.804992,0.486441,0.037307,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.049495008, -0.07142061, -0.005828036, -0.01...","[-0.051018845, 0.013300572, 0.061036862, -0.02...","[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554024,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.05821279, -0.058210686, 0.13881628, -0.0406...",-0.322156,-0.457259,"[-0.044932816, -0.0038222447, 0.08252808, 0.01...",-0.869811,-0.614875,-0.440298,-0.346036,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.056813803, -0.089675784, -0.044186905, 0.0...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
554025,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.05821279, -0.058210686, 0.13881628, -0.0406...",-0.331055,-0.457259,"[-0.044932816, -0.0038222447, 0.08252808, 0.01...",-0.869811,-0.614875,-0.440298,-0.346036,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.13859886, -0.060899...","[-0.056813803, -0.089675784, -0.044186905, 0.0...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
554026,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.05821279, -0.058210686, 0.13881628, -0.0406...",-0.332539,-0.457259,"[-0.044932816, -0.0038222447, 0.08252808, 0.01...",-0.869811,-0.614875,-0.440298,-0.346036,"[-0.032043114, -0.05257272, 0.04719335, -0.044...","[0.07354073, -0.0826873, 0.

In [20]:
import pandas as pd
import numpy as np
import ast # Import the Abstract Syntax Trees module
from warnings import warn # Import warn for issuing warnings

# Assume 'positives' is your pandas positives containing the string representations of vectors

# List of columns that are expected to contain string representations of vectors
# Make sure this list is accurate based on your positives's columns
vector_string_cols = [
    'SOURCE_LANG_task',
    'TARGET_LANG_task',
    'INDUSTRY_task',
    'SOURCE_LANG_EMBED_translator',
    'TARGET_LANG_EMBED_translator',
    'INDUSTRY_EMBED_translator',
    'task_categorical_vector',      # The new condensed columns
    'translator_categorical_vector', # The new condensed columns
]

# Iterate through the identified columns and apply the conversion
for col in vector_string_cols:
    if col in positives.columns: # Check if the column exists in the positives
        print(f"Attempting to convert column '{col}' from string to list/array...")
        # Check if the column is of object dtype, which is typical for strings or mixed types
        if positives[col].dtype == 'object':
            try:
                # Use a lambda function with ast.literal_eval to convert each string cell
                # Add checks within the lambda:
                # 1. Check if the value is a string and not NaN (using pd.notna)
                # 2. Use ast.literal_eval to parse the string
                # 3. Convert the result to a NumPy array with float32 dtype
                # Handle cases where the value might already be a list/array (if conversion was partially done)
                positives[col] = positives[col].apply(
                    lambda x: np.array(ast.literal_eval(x), dtype=np.float32)
                    if isinstance(x, str) and pd.notna(x)
                    else (np.array(x, dtype=np.float32) if isinstance(x, (list, np.ndarray)) else x) # Keep existing arrays/lists, pass others through
                )
                print(f"Conversion successful for column '{col}'.")
            except Exception as e:
                # Catch potential errors during parsing (e.g., malformed string)
                warn(f"Warning: Could not convert column '{col}' from string to list/array. "
                     f"There might be problematic values. Error: {e}")
                # You might want to inspect the problematic rows or column content here
        else:
            print(f"Column '{col}' is not of object dtype ({positives[col].dtype}). Skipping string conversion.")
            # If it's already a numeric type, it's not a string vector.

    else:
        warn(f"Warning: Column '{col}' not found in the positives. Skipping.")

Attempting to convert column 'SOURCE_LANG_task' from string to list/array...
Conversion successful for column 'SOURCE_LANG_task'.
Attempting to convert column 'TARGET_LANG_task' from string to list/array...
Conversion successful for column 'TARGET_LANG_task'.
Attempting to convert column 'INDUSTRY_task' from string to list/array...
Conversion successful for column 'INDUSTRY_task'.
Attempting to convert column 'SOURCE_LANG_EMBED_translator' from string to list/array...
Conversion successful for column 'SOURCE_LANG_EMBED_translator'.
Attempting to convert column 'TARGET_LANG_EMBED_translator' from string to list/array...
Conversion successful for column 'TARGET_LANG_EMBED_translator'.
Attempting to convert column 'INDUSTRY_EMBED_translator' from string to list/array...
Conversion successful for column 'INDUSTRY_EMBED_translator'.
Attempting to convert column 'task_categorical_vector' from string to list/array...
Conversion successful for column 'task_categorical_vector'.
Attempting to co

### To Tensors

In [21]:
from torch.utils.data import Dataset, DataLoader

class PositivesDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

        self.pair_labels = self.dataframe.index.tolist()
        self.samples = []

        for i in range(len(dataframe)):
            self.samples.append((i, self.pair_labels[i], 0))
            self.samples.append((i, self.pair_labels[i], 1))

        self.task_features = [
            'SOURCE_LANG_task',
            'TARGET_LANG_task',
            'INDUSTRY_task',
            'FORECAST_task',
            'HOURLY_RATE_task',
            'task_categorical_vector',
        ]

        self.translator_features = [
            'SOURCE_LANG_EMBED_translator',
            'TARGET_LANG_EMBED_translator',
            'INDUSTRY_EMBED_translator',
            'HOURS_mean',
            'HOURLY_RATE_mean',
            'HOURLY_RATE_translator',
            'QUALITY_EVALUATION_mean',
            'translator_categorical_vector',
        ]

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        original_row_idx, label, entity_type = self.samples[idx]
        row_data = self.dataframe.iloc[original_row_idx]

        if entity_type == 0:
            feature_list = []
            for col in self.task_features:
                col_data = row_data[col]
                if isinstance(col_data, (list, np.ndarray)):
                    feature_list.extend(col_data)
                elif isinstance(col_data, (int, float, np.number)):
                    feature_list.append(col_data)
                else:
                    print(col_data)

            features = np.array(feature_list, dtype=np.float32)

        elif entity_type == 1:
            feature_list = []
            for col in self.translator_features:
                col_data = row_data[col]
                if isinstance(col_data, (list, np.ndarray)):
                    feature_list.extend(col_data)
                elif isinstance(col_data, (int, float, np.number)):
                    feature_list.append(col_data)
                else:
                    print(type(col_data))
            features = np.array(feature_list, dtype=np.float32)

        else:
            raise ValueError("entity_type must be 0 or 1")
        
        features_tensor = torch.from_numpy(features)
        label_tensor = torch.tensor(label, dtype=torch.float32)
        entity_type_tensor = torch.tensor(entity_type, dtype=torch.float32)

        return features_tensor, label_tensor, entity_type_tensor

def collate_fn(batch):
    return tuple(zip(*batch))

In [22]:
dataset = PositivesDataset(positives)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

dataloader_iter = iter(dataloader)
batch = next(dataloader_iter)
features, labels, entity_types = batch
print(features[0].shape)
print(features[1].shape)

torch.Size([50])
torch.Size([48])


### Save DataFrame

In [23]:
positives.to_pickle('../data/final_positives.pkl')

In [24]:
# ensure that the tensors have the same size
for data in dataloader:
    features, labels, entity_types = data
    assert features[0].shape == features[1].shape, "Features tensors have different shapes"
    assert labels[0].shape == labels[1].shape, "Labels tensors have different shapes"
    assert entity_types[0].shape == entity_types[1].shape, "Entity types tensors have different shapes"

AssertionError: Features tensors have different shapes